In [ ]:
#As always, we import everything
import os
import pandas as pd
import json
import folium
import math
%matplotlib inline
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import base64

We define a RESET constant, that allows the recreation of all graphs and all mappings. We will detail later in the notebook its use.

In [ ]:
RESET = False

First, let's get the data and put it in a dataframe.
The .tsv file was found on the eurostat website. We modified it RADEK EXPLIQUE

In [ ]:
eu_data = pd.read_table('tsdec450.tsv', na_values=': ')

Now, we can process it to get the id for each country.

In [ ]:
eu_data['info'] = eu_data['info'].apply(lambda x: x.split(",")[3])
eu_data

Now we can define a function that returns the color of a country based on its unemployement rate.
For this, we need to choose a palette of colors : we used colorbrewer to get one which is color-blind friendly.
To get the color for a country, we simply bin all the unemployement_rate into one of the colors.

We decided to use a linear scale from the lowest unemployement value to the highest in our data.

Later on, we will need to get the full name for each country. We chose to create the mapping from country id to country name, as both the id and the name are available in the topoJSON values. This mapping is only recreated if the RESET constant is set to True. Otherwise, we will load it from a JSON file.

In [ ]:
colors_unemployement = ['#fef0d9','#fdd49e','#fdbb84','#fc8d59','#e34a33','#b30000']
min_unemployement_eu = eu_data['2016'].min()
max_unemployement_eu = eu_data['2016'].max()

id_country_mapping = {}

def get_color_eu(country, properties):
    if RESET:
        global id_country_mapping
        id_country_mapping[country] = properties['NAME']
    
    values = eu_data.loc[eu_data['info'] == country, '2016'].values
    if len(values) == 0:
        return '#000000'
    
    unemployement_rate = values[0]
    ratio = (unemployement_rate - min_unemployement_eu) / (max_unemployement_eu - min_unemployement_eu)
    index = math.floor(ratio * len(colors_unemployement))
    if index == len(colors_unemployement):
        index = index - 1
    return colors_unemployement[index]

We also define a function that checks if a country is in the dataframe, and if not, we make its overly transparent.

In [ ]:
def get_opacity_eu(country):
    values = eu_data.loc[eu_data['info'] == country, '2016'].values
    return 1 if len(values) > 0 else 0

Now we can create the Europe map, and add the overlay, using the previous functions.

The second map is only used to display the legend.
We used to following website to convert the topojson file to gejson
https://jeffpaine.github.io/geojson-topojson/

In [ ]:
map_eu = folium.Map([51,15], tiles='cartodbpositron', zoom_start=4)


# Color of the country
folium.TopoJson(
    open('topojson/europe.topojson.json'),
    object_path='objects.europe',
    style_function=lambda feature: {
        'fillOpacity' : get_opacity_eu(feature['id']), #opacity for the fill color
        'opacity' : get_opacity_eu(feature['id']), #opacity for the borders
        'fillColor': get_color_eu(feature['id'], feature['properties']),
        'color' : 'black',
        'weight' : 1
        }
).add_to(map_eu)

# 
"""map_eu.choropleth(geo_data='topojson/europe.geojson.json',data=eu_data,
             columns=['info', '2016'],
             key_on='feature.id',
             fill_color='OrRd',
             #fill_color='RdGy',
             fill_opacity=0, 
             line_opacity=0.0,
             legend_name='Percentage of unemployement in country')"""

For interactivity, we decided to place, for each country a popup displaying the graph of unemployement from 1990 to 2016. Folium provides a way to add a marker, that when clicked on, displays a popup.

Thus, we first need to get, for each country its center to place the marker at the right coordinates.
We used the geopy library to do this.

As this operation is quite slow, and not 100% accurate for each country, we decided to create a mapping using the library, then save it to a JSON, and finally to manually edit this JSON if the coordinates were not accurate.
When this was done, we only need to reload this JSON, not recreate it each time the notebook is run.

In [ ]:
if RESET:
    position_mapping = {}
    geolocator = Nominatim()
    for country_id in eu_data['info']:
        if country_id in id_country_mapping:
            location = geolocator.geocode(id_country_mapping[country_id])
            if (location):
                position_mapping[country_id] = [location.latitude,location.longitude]

    with open('position_mapping.json', 'w') as outfile:
        json.dump(position_mapping, outfile)

    with open('id_country_mapping.json', 'w') as outfile:
        json.dump(id_country_mapping, outfile)
else:
    with open('position_mapping.json', 'r') as infile:
        position_mapping = json.load(infile)

    with open('id_country_mapping.json', 'r') as infile:
        id_country_mapping = json.load(infile)

Now that we have the positions, for each country we create a graph and save it. We used the previously defined id_country_mapping to get the full name of each country.

In [ ]:
for index, row in eu_data.iterrows():
    global id_country_mapping
    
    if row.values[0] in id_country_mapping:
        row_values = row.iloc[1:].astype(float)
        plot = row_values.transpose().plot()
        plt.ylim(0,30)
        plt.xlabel('Year')
        plt.ylabel('% of unemployed')
        plt.title('Graph of unemployment in ' + id_country_mapping[row.values[0]])
        plt.savefig('eu_graphs/graph_' + row.values[0] + '.png')
        plt.close()

We decided to host all the graphs on our personal server, in order to keep the maps at a low size.

Now that we have both the graphs and the positions, we can, for each country, place a custom marker (a black disk) that displays the corresponding graph when clicked. 

In [ ]:
all_markers = folium.FeatureGroup("Markers")
for country_id in eu_data['info']:
    if country_id in position_mapping:
        location = position_mapping[country_id]
        if location:
            html = '<img src="http://feudal-ambitions.com/ADA/HW3/eu_graphs/graph_' + country_id + '" width=450 height=300>'
            iframe = folium.IFrame(html, width=470, height=320)
            popup = folium.Popup(iframe, max_width=2650)

            custom_icon = folium.features.CustomIcon("http://www.i2symbol.com/images/symbols/geometry/black_circle_u25CF_icon_256x256.png", icon_size=(25, 25))
            all_markers.add_child(folium.Marker(location, popup=popup, icon=custom_icon))


map_eu.add_child(all_markers)
map_eu

In [ ]:
#ch_data_raw = pd.read_csv('data/unemployed_switzerland.csv')
ch_data = pd.read_csv('data/unemployed_switzerland.csv')

In [ ]:
"""groups = ch_data_raw[['Year', 'Canton', 'Unemployment Rate']].groupby(['Canton'])
cantons = ch_data_raw['Canton'].unique()
years = ch_data_raw['Year'].unique()
ch_data = pd.DataFrame(index = cantons, columns=years)

for group, data in groups:
    ch_data.loc[group] = data['Unemployment Rate'].values
"""
ch_data

In [ ]:
colors_unemployement = ['#fef0d9','#fdd49e','#fdbb84','#fc8d59','#e34a33','#b30000']
min_unemployement_ch = ch_data['2017'].min()
max_unemployement_ch = ch_data['2017'].max()

id_canton_mapping = {}

def get_color_ch(canton, properties):
    global id_canton_mapping
    id_canton_mapping[canton] = properties['name']
    
    values = ch_data.loc[ch_data['Canton'] == canton, '2017'].values
    if len(values) == 0:
        return '#000000'
    
    unemployement_rate = values[0]
    ratio = (unemployement_rate - min_unemployement_ch) / (max_unemployement_ch - min_unemployement_ch)
    index = math.floor(ratio * len(colors_unemployement))
    if index == len(colors_unemployement):
        index = index - 1
    return colors_unemployement[index]

In [ ]:
map_ch = folium.Map([46.8,8.3], tiles='cartodbpositron', zoom_start=8)


# Color of the country
folium.TopoJson(
    open('topojson/ch-cantons.topojson.json'),
    object_path='objects.cantons',
    style_function=lambda feature: {
        'fillOpacity' : 1,#get_opacity_eu(feature['id']), #opacity for the fill color
        'opacity' : 1,#get_opacity_eu(feature['id']), #opacity for the borders
        'fillColor': get_color_ch(feature['id'], feature['properties']),
        'color' : 'black',
        'weight' : 1
        }
).add_to(map_ch)

In [ ]:
"""position_canton_mapping = {}
geolocator = Nominatim()
for canton_id in id_canton_mapping:
    location = geolocator.geocode(id_canton_mapping[canton_id])
    if (location):
        position_canton_mapping[canton_id] = [location.latitude,location.longitude]

with open('position_canton_mapping.json', 'w') as outfile:
    json.dump(position_canton_mapping, outfile)
    
with open('id_canton_mapping.json', 'w') as outfile:
    json.dump(id_canton_mapping, outfile)"""

with open('position_canton_mapping.json', 'r') as infile:
    position_canton_mapping = json.load(infile)
    
with open('id_canton_mapping.json', 'r') as infile:
    id_canton_mapping = json.load(infile)

In [ ]:
for index, row in ch_data.iterrows():
    global id_canton_mapping
    
    if row.values[0] in id_canton_mapping:
        row_values = row.iloc[1:].astype(float)
        plot = row_values.transpose().plot()
        plt.ylim(0,30)
        plt.xlabel('Year')
        plt.ylabel('% of unemployed')
        plt.title('Graph of unemployment in ' + id_canton_mapping[row.values[0]])
        plt.savefig('ch_graphs/graph_' + row.values[0] + '.png')
        plt.close()

In [ ]:
all_markers = folium.FeatureGroup("Markers")
for canton_id in id_canton_mapping:
    location = position_canton_mapping[canton_id]
    
    html = '<img src="http://feudal-ambitions.com/ADA/HW3/ch_graphs/graph_' + canton_id + '" width=450 height=300>'
    iframe = folium.IFrame(html, width=470, height=320)
    popup = folium.Popup(iframe, max_width=2650)

    custom_icon = folium.features.CustomIcon("http://www.i2symbol.com/images/symbols/geometry/black_circle_u25CF_icon_256x256.png", icon_size=(25, 25))
    all_markers.add_child(folium.Marker(location, popup=popup, icon=custom_icon))
        


map_ch.add_child(all_markers)
map_ch

In [ ]:
ch_data2 = pd.read_csv('data/unemployed_num_switzerland.csv', thousands="'")
ch_data3 = pd.read_csv('data/jobseekers_switzerland.csv', thousands="'")
ch_data4 = pd.read_csv('data/unemployed_foreigner_swiss.csv', thousands="'")

In [ ]:
active_pop = ((ch_data2.iloc[:, 1:] * 100) / ch_data.iloc[:, 1:])
ch_jobseeker_rate = (ch_data3.iloc[:, 1:] * 100) / active_pop
ch_jobseeker_rate = pd.concat((ch_data['Canton'], ch_jobseeker_rate), axis=1)

In [ ]:
active_pop.to_csv('data/pop_canton.csv')

In [ ]:
colors_unemployement = ['#fef0d9','#fdd49e','#fdbb84','#fc8d59','#e34a33','#b30000']
min_jobseeker_ch = ch_jobseeker_rate['2017'].min()
max_jobseeker_ch = ch_jobseeker_rate['2017'].max()

def get_color_ch_j(canton, properties):
    values = ch_jobseeker_rate.loc[ch_jobseeker_rate['Canton'] == canton, '2017'].values
    if len(values) == 0:
        return '#000000'
    
    jobseeker_rate = values[0]
    ratio = (jobseeker_rate - min_jobseeker_ch) / (max_jobseeker_ch - min_jobseeker_ch)
    index = math.floor(ratio * len(colors_unemployement))
    if index == len(colors_unemployement):
        index = index - 1
    return colors_unemployement[index]

In [ ]:
map_ch_j = folium.Map([46.8,8.3], tiles='cartodbpositron', zoom_start=8)


# Color of the country
folium.TopoJson(
    open('topojson/ch-cantons.topojson.json'),
    object_path='objects.cantons',
    style_function=lambda feature: {
        'fillOpacity' : 1,#get_opacity_eu(feature['id']), #opacity for the fill color
        'opacity' : 1,#get_opacity_eu(feature['id']), #opacity for the borders
        'fillColor': get_color_ch_j(feature['id'], feature['properties']),
        'color' : 'black',
        'weight' : 1
        }
).add_to(map_ch_j)

In [ ]:
for index, row in ch_jobseeker_rate.iterrows():
    global id_canton_mapping
    row_values = row.iloc[1:].astype(float)
    plot = row_values.transpose().plot()
    plt.ylim(0,30)
    plt.xlabel('Year')
    plt.ylabel('% of unemployed')
    plt.title('Graph of unemployment in ' + id_canton_mapping[row.values[0]])
    plt.savefig('ch_graphs/graph_j_' + row.values[0] + '.png')
    plt.close()
ch_jobseeker_rate

In [ ]:
all_markersj = folium.FeatureGroup("Markers")
for canton_id in id_canton_mapping:
    location = position_canton_mapping[canton_id]
    
    html = '<img src="http://feudal-ambitions.com/ADA/HW3/ch_graphs/graph_j_' + canton_id + '" width=450 height=300>'
    iframe = folium.IFrame(html, width=470, height=320)
    popup = folium.Popup(iframe, max_width=2650)

    custom_icon = folium.features.CustomIcon("http://www.i2symbol.com/images/symbols/geometry/black_circle_u25CF_icon_256x256.png", icon_size=(25, 25))
    all_markersj.add_child(folium.Marker(location, popup=popup, icon=custom_icon))

map_ch_j.add_child(all_markersj)
map_ch_j

In [ ]:
ch_data4
ch_foreigner = ch_data4[ch_data4.Nationalité == 'Etrangers']
ch_foreigner.index = range(0,len(ch_foreigner))
ch_foreigner = ch_foreigner.drop('Nationalité', axis=1)
ch_swiss = ch_data4[ch_data4.Nationalité == 'Suisses']
ch_swiss.index = range(0,len(ch_swiss))
ch_swiss = ch_swiss.drop('Nationalité', axis=1)



In [ ]:
ch_swiss_rate = (ch_swiss.iloc[:, 1:] * 100) / active_pop
ch_swiss_rate = pd.concat((ch_data['Canton'], ch_swiss_rate), axis=1)
ch_foreigner_rate = (ch_foreigner.iloc[:, 1:] * 100) / active_pop
ch_foreigner_rate = pd.concat((ch_data['Canton'], ch_foreigner_rate), axis=1)



In [ ]:
ch_swiss

As always, we decided to show the graphs of unemployement, this time, for both Swiss and foreigners.

In [ ]:
for index, row in ch_swiss_rate.iterrows():
    global id_canton_mapping
    row_values = row.iloc[2:].astype(float)
    plot = row_values.transpose().plot()
    row_values = ch_foreigner_rate.iloc[index][2:].astype(float)
    plot = row_values.transpose().plot()
    plt.ylim(0,10)
    plt.legend(['Swiss', 'Foreigners'])
    plt.xlabel('Year')
    plt.ylabel('% of unemployed')
    plt.title('Graph of unemployment in ' + id_canton_mapping[row.values[0]])
    plt.savefig('ch_graphs/graph_s_f_' + row.values[0] + '.png')
    plt.close()
    


In order to display the most informations in a single map, we decided to show small bar plots for each canton, showing the Swiss and the foreigners unemployement rates, as well as a color for each canton, indicating the difference.

First, we need to create these small bar plots, and save them. The produced graphs are then hosted on our personal server.

In [ ]:
for index, row in ch_swiss_rate.iterrows():
    global id_canton_mapping
    row_values = [row.iloc[-1], ch_foreigner_rate.iloc[index][-1]]
    plot = plt.bar([0,1], row_values, edgecolor='black',linewidth=10)
    plot[0].set_color('b')
    plot[0].set_linewidth(30)
    plot[1].set_color('orange')
    
    plt.ylim(0,6)
    plt.axis('off')
    plt.savefig('icon_ch/icon_' + row.values[0] + '.png')
    plt.close()

Now, we can write the function that outputs a color based on the difference between the two unemployement rates. We decided to use two linear scale, on for the negative values, and one for the positive ones. This is done to ensure that white values correspond to a small difference, and colorful ones for large difference either positive of negative.

In [ ]:
colors_s_f_neg = ['#b2182b','#d6604d','#f0b572','#fddbc7']
colors_s_f_pos = ['#d1e5f0','#92c5de','#4393c3','#2166ac']

diff_ch_for = ch_swiss_rate.drop('Canton', axis=1).subtract(ch_foreigner_rate.drop('Canton', axis=1))
diff_ch_for = pd.concat((ch_data['Canton'], diff_ch_for), axis=1)


min_s_f_ch = diff_ch_for['2017'].min()
max_s_f_ch = diff_ch_for['2017'].max()

def get_color_ch_s_f(canton, properties):
    diff_ch_s_f = diff_ch_for.loc[diff_ch_for['Canton'] == canton, '2017'].values
    if (diff_ch_s_f < 0):
        ratio = (diff_ch_s_f - min_s_f_ch) / (-min_s_f_ch)
        colors = colors_s_f_neg
    else:
        ratio = (diff_ch_s_f) / (max_s_f_ch)
        colors = colors_s_f_pos

        
    index = math.floor(ratio * len(colors))
    if index == len(colors):
        index = index - 1
    return colors[index]

As always, we can now create the map.

In [ ]:
map_ch_s_f = folium.Map([46.8,8.3], tiles='cartodbpositron', zoom_start=8)


# Color of the country
folium.TopoJson(
    open('topojson/ch-cantons.topojson.json'),
    object_path='objects.cantons',
    style_function=lambda feature: {
        'fillOpacity' : 1
        'opacity' : 1
        'fillColor': get_color_ch_s_f(feature['id'], feature['properties']),
        'color' : 'black',
        'weight' : 1
        }
).add_to(map_ch_s_f)

And add the 'markers' (that are the small bar plots), that display the graphs.

In [ ]:
all_markers_s_f = folium.FeatureGroup("Markers")
for canton_id in id_canton_mapping:
    location = position_canton_mapping[canton_id]
    
    html = '<img src="http://feudal-ambitions.com/ADA/HW3/ch_graphs/graph_s_f_' + canton_id + '" width=450 height=300>'
    iframe = folium.IFrame(html, width=470, height=320)
    popup = folium.Popup(iframe, max_width=2650)

    custom_icon = folium.features.CustomIcon("http://feudal-ambitions.com/ADA/HW3/icon_ch/icon_"+ canton_id +".png", icon_size=(30, 60))
    all_markers_s_f.add_child(folium.Marker(location, popup=popup, icon=custom_icon))

map_ch_s_f.add_child(all_markers_s_f)
map_ch_s_f